In [2]:
import pandas as pd 
import numpy as np
import re


In [3]:
# Read data from file 'filename.csv' 
# (in the same directory that your python process is based)
# Control delimiters, rows, column names with read_csv (see later)
df_list=[]
file_list=[r'./input/dummy_data.csv']
for f in file_list:
    dftemp = pd.read_csv(f) 
    df_list.append(dftemp)
df=pd.concat(df_list)
df.shape

FileNotFoundError: [Errno 2] File ./input/dummy_data.csv does not exist: './input/dummy_data.csv'

In [ ]:
df.head()

We do some cleanup, typically files may come with nulls or NAs

In [18]:
df['article_sentences'].fillna("",inplace=True)
df['article_sentences'] = df['article_sentences'].str.replace('\@','',regex=True)

This is where we define the list of keywords, if we have a large set, we can keep them in a text file or excel file  
and load it into a list  
We use here regular expressions, which is a powerful pattern matching library/language.


In [19]:
re_list=['.*scandal.*','.*analytics.*','.*continuous.*','.*statistical.*']

In [20]:
#%%time
def re_search(re_list,df, field_name):
    n=1
    for re_text in re_list:
        r = re.compile(re_text,re.IGNORECASE)
        regmatch = np.vectorize(lambda x: bool(r.match(x)))
        df["re_match"+str(n)]=regmatch(df[field_name].values)
        n=n+1
re_search(re_list,df,"article_sentences")

In [21]:
df[df['re_match1']==True]['article_sentences']

194     Regulation surrounding entity-level controls =...
801     The Big Five became the Big Four after the dem...
827     One result of this scandal was that Arthur And...
828     A recent trend in audits (spurred on by such a...
1351    2002: Sarbanes-Oxley Act is passed in the wake...
1557    High-profile commercial scandals and failures ...
Name: article_sentences, dtype: object

In [22]:
df.head()

,Unnamed: 0,page_title,article_sentences,page_url,re_match1,re_match2,re_match3,re_match4,kw_match1,kw_match2,kw_match3,kw_match4
0,0,News analytics,News analysis refers to the measurement of the...,https://en.wikipedia.org/wiki/News_analytics,False,False,False,False,False,False,False,False
1,1,News analytics,"Some of these attributes are: sentiment, relev...",https://en.wikipedia.org/wiki/News_analytics,False,False,False,False,False,False,False,False
2,2,News analytics,Expressing news stories as numbers and metadat...,https://en.wikipedia.org/wiki/News_analytics,False,False,False,True,False,False,False,True
3,3,News analytics,This data is often used in financial markets a...,https://en.wikipedia.org/wiki/News_analytics,False,False,False,False,False,False,False,False
4,4,News analytics,News analytics are usually derived through aut...,https://en.wikipedia.org/wiki/News_analytics,False,True,False,False,False,True,False,False


In [11]:
keyword_list=['scandal','analytics','continuous','statistical']

In [13]:
#%%time
def keyword_search(keyword_list,df, field_name):
    keyword_list=[x.lower() for x in keyword_list]
    listed = df[field_name].str.lower().tolist()
    n=1
    for kw in keyword_list:
        print(kw)
        df["kw_match"+str(n)] = [kw in n for n in listed]
        n=n+1
    return None
keyword_search(keyword_list,df,'article_sentences')


scandal
analytics
continuous
statistical


In [14]:
df[df['kw_match1']==True]['article_sentences']

194     Regulation surrounding entity-level controls =...
801     The Big Five became the Big Four after the dem...
827     One result of this scandal was that Arthur And...
828     A recent trend in audits (spurred on by such a...
1351    2002: Sarbanes-Oxley Act is passed in the wake...
1557    High-profile commercial scandals and failures ...
Name: article_sentences, dtype: object

In [15]:
df_output=df[df[['re_match1','re_match2',"re_match3","re_match4"]].any(1)]

In [16]:
df_output.to_excel("./output_data/found_records.xlsx")